# Part 2 - Gauge Freedom and Symmetries

In [1]:
#widget has bugs with reloading plot
#%matplotlib widget
%matplotlib inline
#TODO: widget works, but size is not displayed correctly

import numpy as np 
from scipy.sparse import diags # Used for banded matrices

import matplotlib as mpl
import matplotlib.pyplot as plt # Plotting
from cycler import cycler


import seaborn as sns
plt.style.use('seaborn-dark')
plt.rcParams.update({'font.size':14})
#mpl.rcParams['figure.dpi']= 100

from IPython.display import display, Markdown, Latex, clear_output # used for printing Latex and Markdown output in code cells
from ipywidgets import Layout, fixed, HBox, VBox #interact, interactive, interact_manual, FloatSlider, , Label, Layout, Button, VBox
import ipywidgets as widgets

import functools
import time, math
from scipy.linalg import expm 


%load_ext autoreload
%autoreload 2

  
# adding `Modules/` to the system path
import sys
sys.path.insert(0, 'Modules/')

# Gauge Freedom
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mela}[3]{\left\langle #1 \vphantom{#2#3} \right| #2 \left| #3 \vphantom{#1#2} \right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$
$\renewcommand{\vec}{\mathbf}$


A wavefunction $\ket{\psi}$ is not observable and thus not necessarily unique. One can add an arbitrary phase to each orbital of our $n$-chain, which would change the wavefunction, but not the underlying physics, as probabilities of an outcome $O_i$ are calculated using the norm of the projection of the probed wavefunction onto the $i$-th eigenstate of the operator $|\braket{\phi_i}{\psi}|^2$. The same holds for Operators, whose representation depends on the chosen picture. Observables however are unique and correspond to the eigenvalues of a hermitian operator $\hat O $.

In the following the phase $\exp(\ii \theta_{ij})$ accumulated for hopping from one orbital $i$ to another $j$ can be changed. Observe how on the one hand the matrix elements of the Hamiltonian $H_{ij}$ change, but on the other hand the eigenvalues (and thus the possible observables) stay the same. The values of $\theta_{ij}$ are calculated using a `phase` vector as input
\begin{equation}
    \theta_{ij} = \mathrm{phase}[i] - \mathrm{phase}[j] .
\end{equation}
This guarantees that no matter which path we take from any given initial position back to the same position, we will always end up with a net phase of zero and thus preserve the structure of the Hamiltonian.

The freedom to redefine the structure and fix redundant degrees of freedom in our theory (in our case the change of a - not observable - phase for each orbital) without changing the underlying physics is commonly called **Gauge Freedom**. Another example from [classical electromagnetism](https://en.wikipedia.org/wiki/Gauge_theory#Classical_electromagnetism) would be the gauge freedom to add a gradient of a (twice continuously differentiable) scalar function $\nabla f$ to the vector potential $\vec A$, which do not change the observables, namely the electric- and magentic fields $\vec E$ and $\vec B$, respectively.

In [2]:
#TODO add dropdwon to display different hamiltionains with their respective eigenvalues and explain the phases
#TODO possibly add slider to change number of decimal digets diplayed
from Module_Widgets_and_Sliders import phase_Text_Box, n_Slider, phase_Dropdown, precision_Slider
from Module_Symmetry_and_Gauge import Hopping_Matrix_with_Phase, Show_Hamiltonian_Gaugefreedom

In [3]:
#TODO: add description of what happenw ith pahse exp(i) everywhere
# TODO: chekc that pahse 0 teturn a real hamiltonian

H_phase = widgets.interactive(Show_Hamiltonian_Gaugefreedom,
            phase=phase_Dropdown,           
            n=n_Slider,
            precision=precision_Slider);


display(VBox([phase_Text_Box, H_phase]))

TODO: add theory to magnetic flux and matrix
TODO: add slider and widgets to play around with the phase
TODO: add default phases

In [4]:
from Module_Symmetry_and_Gauge import Magnetic_Flux_Matrix, Show_Magnetic_Flux
#TODO: possibly add more than just exp(1j) as phase

In [5]:
M_widget = widgets.interactive(Show_Magnetic_Flux,           
            n=n_Slider,
            precision=precision_Slider);


display(M_widget)

interactive(children=(BoundedIntText(value=6, description='$n = $', layout=Layout(width='3cm'), max=10, min=2,…

# Symmetry

Let us focus on one of the most important and fundamental concepts in the entire field of physics: **Symmetry**. Local and global symmetries are present in all modern theories of physics (QED, QCD, QFT, GR, the Standard Model, etc.) and lead to interesting concepts such as gauge freedom and conservation laws via Noether's theorem and are present in things as simple as coordinate transformations. 

But what is a symmetry of an object? Plainly speaking, if an object is invariant (i.e. stays the same) after applying some transformation, this action is called a symmetry operation. As a simple example consider as object a square. The transformaton of rotating it by either $\frac{pi}{2}, \pi, \frac{3\pi}{2}$ or $2\pi$ is a symmetry operation.

## Groups
The underlying mathematical structure that deals (among other things) with all symmetry operations of an object is called the object's [symmetry group](https://en.wikipedia.org/wiki/Symmetry_group#One_dimension). Mathematically, a [group](https://en.wikipedia.org/wiki/Group_(mathematics)) is a set $G$ with a binary operation $\circ$ such that:

1. Closure: Every combination of elements from $G$ is again contained in $G$
    $$ a \circ b \in G \quad \forall a, b \in G.$$
    
2. Associativity:
    $$ (a \circ b) \circ c = a \circ (b \circ c) \quad \forall a, b, c \in G.$$
    
3. Identity: There exists a neutral element $e \in G$, such that
    $$ e \circ a = a = a \circ e \quad \forall a \in G.$$
    
4. Inverse: For each element $a \in G$ there exists an (unique) inverse element, denoted $a^{-1} \in G$, such that
    $$ a \circ a^{-1} = e = a^{-1} \circ a \quad \forall a \in G.$$

## Symmetry in QM
### Invariance of $H$
Now that we know the precise definiton of a group, we can specify symmetry in QM. First, note that a symmetry operation must leave the norm of a wave vector invariant. Thus leaving us with unitary operators ($\hat{O} \hat{O}^\dagger = \mathbb{1}$). A unitary operator $O$ that encapsulates a symmetry with the Hamiltonian $\hat{H}$ is defined by vanishing commutator, i.e.
    $$ [\hat{H}, \hat{O}] = 0,$$
where $[\hat{A}, \hat{B}] \coloneqq \hat{A} \hat{B} - \hat{B} \hat{A}$ denotes the commutator of two operators. The reason for symmetry being connected with vanishing commutator becomes apparant by considering the inverse of a unitary operator is just its conjugate transpose, i.e. $\hat{O}^{-1} = \hat{O}^\dagger$ and thus
$$
    [O,H] = 0 \implies OH = HO. 
$$
Multiplying with the inverse of $O$ from the right thus gives
$$
    OHO^{-1} = OHO^\dagger = HOO^{-1} = H.
$$
which implies that $H$ is invariant under the linear transfomation $OHO^\dagger$ induced by $O$.

### Common eigen basis
Another very important property of commuting operators is, that they have a common eigenbasis, meaning that every eigenvector of the one operator is also an eigenvector of the other operator. The proof is straight forward and we will perform it for the Hamilton operator:

Assume $\{E_n\}$ and $\{\ket{\psi_n}\}$ are the sets of eigenvalues and corresponding eigenvectors of the Hamiltonian, respectively. Thus
    $$ H \ket{\psi_n} = E_n \ket{\psi_n},$$
    
holds. Acting with an operator $T$ that commutes with $H$, $[H, T] = 0$ on the eigenvalue equation gives
    $$  H (T\ket{\psi_n})  = T (H \ket{\psi_n}) = T (E_n \ket{\psi_n}=  E_n (T \ket{\psi_n}), $$

implying that the vector $T\ket{\psi_n}$ is also an eigenvector of the Hamiltonian. Assuming non-degenerate eigenvalues because $T\ket{\psi_n}$ and $\ket{\psi_n}$ are both eingevectors of $H$ they can differ only by a multiplicative constant $t_n$. This implies 
$$ T\ket{\psi_n} = t_n \ket{\psi_n}, $$
proving that $T$ and $H$ share a common set of eigenvectors.

Note: This also holds for degnerete eigenvalues, but the proof is a bit more subtile.

The gist of all this is that we can apply a symmetry operation to our eigenvectors and are guranteed to get back an eigenvector (or at least a linear combination of eigenvectors if degenerate eigenvalues are present) of $H$

### Cyclic group $C_n$ / Abelian group of translations

The $n$-chain model consists of a special type of symmetry group, the [cyclic group $C_n$](https://en.wikipedia.org/wiki/Cyclic_group) which can be represented by $n$ rotations around the origin with angle $\frac{2 \pi m}{n}$ where $m \in \{0, 1, \ldots n-1\}$. This group has another special property in addition to the four group axioms, all group operations are commutative
    $$ a \circ b = b \circ a \quad \forall a, b \in C_n .$$
    
Groups with this property are called abelian groups. Intuitively rotations emit this property as no matter the order you perform two consecutive rotations, the result can be expressed by a rotation with the sum of both rotation angles.

Aditionaly, one can construct all matrix representations of the elements of the group $C_n$ by repeatedly applying a single roation to the right $T$, which results in all the elements $\{\mathbb{1}_n, T, T^2, \ldots, T^{n-1}\}$. The matrix $T \in \mathbb{R}^{n \times n}$ is defined by

$$ T = \begin{pmatrix}
            0 & 1 & 0 & 0 & \cdots \\
            0 & 0 & 1 & 0 & \cdots \\
            \vdots & \vdots & \ddots & \ddots & \vdots\\
            1 & 0 & 0 & 0 & 0
        \end{pmatrix}
$$

In the code below one can look at the different matrices representing rotations from one position to another. Note that the identity element is always the $n$-dimensional unit matrix.

In [6]:
from Module_Widgets_and_Sliders import turns_Slider
from Module_Symmetry_and_Gauge import Translation_Group, Right_Translation_Matrix

In [7]:
translation_widget = widgets.interactive(Right_Translation_Matrix,
                                        n=n_Slider,
                                        turns=turns_Slider,
                                        show=widgets.fixed(True));

display(translation_widget)

interactive(children=(BoundedIntText(value=6, description='$n = $', layout=Layout(width='3cm'), max=10, min=2,…

Note how a left translation $t_\mathrm{l}$ (with negative number of roations) is equivalent to a right translations with $t_\mathrm{r} = t_\mathrm{l} \operatorname{mod} n$, e. g. for $n=6$, two turns to the left, $t_\mathrm{l} = -2 \iff t_\mathrm{r} = 4$ is equal to four right turns

Now let's look at the abelian cyclic group $C_n$ with all its elements

In [8]:
translation_group_widget = widgets.interactive(Translation_Group,
                                        n=n_Slider,
                                        show=widgets.fixed(True));

display(translation_group_widget)

interactive(children=(BoundedIntText(value=6, description='$n = $', layout=Layout(width='3cm'), max=10, min=2,…

Finally we can check that our Hamiltonian $H$ commutes with a right Translation $T$. Interestingly, also the Hamiltonian with magnetic flux, $M$ commutes with $T$. But the Hamiltonian does not commute anymore, if we add an arbitray `phase` vector.

In [9]:
# Ask professot why this is? And also ask if this changes the physics, as the eigenvalues of TH stay the same.

In [10]:
from Module_Symmetry_and_Gauge import Commutator, Show_Commutator

In [11]:
H = Hopping_Matrix_with_Phase
T = Right_Translation_Matrix
M = Magnetic_Flux_Matrix
#TOOO  adjust space evenly for widget output
#TODO think about wheter dispaying the eigenvlaues is usefull for understing the concept
comm_HT_widget = widgets.interactive(Show_Commutator,
                            A=fixed(H),
                            B=fixed(T),
                            name_A=fixed("H"),
                            name_B=fixed("T"),
                            phase=phase_Dropdown,
                            n=n_Slider,
                            precision=precision_Slider,
                            turns=turns_Slider,
                        );

comm_TM_widget = widgets.interactive(Show_Commutator,
                            A=fixed(M),
                            B=fixed(T),
                            name_A=fixed("M"),
                            name_B=fixed("T"),
                            n=n_Slider,
                            precision=precision_Slider,
                            turns=turns_Slider,
                        );

display(HBox([VBox([phase_Text_Box, comm_HT_widget]),
             VBox([widgets.Text(), widgets.Text(), comm_TM_widget])]))

# Reflection Symmetry

Another important symmetry of our $n$-chain is **reflection** symmetry. A regular $n$-gon has $n$ axis of symmetry, which
* are the $n$ lines connecting the midpoints of an edge with its opposite vertex, iff $n$ is odd,

or
* are $n/2$ lines connecting two opposite vertices and another $n/2$ lines connecting the midpoints of opposite edges (also known as perpendicular bisector), iff $n$ is even

To be able to distinguish different vertices and edges we use the following notation:
1. We view each $n$-chain (no matter the parity) hanging as if one vertex was attached to a string and beeing pulled down by gravity. This anker vertex is assigned the number 1 
2. Every other vertex is numbered upwards going clockwise.
3. Each edge is also numbered from 1 to $n$ clockwise starting from the edge on the right of vertex 1.

For example, take this pentagon with an axis of reflection going through vertex 1 and the midpoint of edge 3

<img src="Images_For_Notebooks/pentagon_symmetry_line.png" width=200 />
<figcaption> Fig: 1D NN-Chain with 8 sites and a particle (blue) on site 1</figcaption>

TODO: change figure caption and add hexagon with symmetry lines adn caption of `axis` parameter

In the code below one can look at different reflection matrices, where the axis of reflection can be chosen via the `axis` parameter, which starts from 1 representing a reflection along the vertical line going through vertex 1. Incrementing `axis` turns the axis of rotation clockwise to the right onto the next symmetry line.

In [12]:
from Module_Symmetry_and_Gauge import Reflection_Matrix, All_Reflections
from Module_Widgets_and_Sliders import axis_Slider

reflection_widget = widgets.interactive(Reflection_Matrix,
                                        n=n_Slider,
                                        axis=axis_Slider,
                                        show=widgets.fixed(True));

display(reflection_widget)

interactive(children=(BoundedIntText(value=6, description='$n = $', layout=Layout(width='3cm'), max=10, min=2,…

Now we can check if reflections commute with $H$ or the magnetic flux Hamiltonian $M$. Turns out, reflections do not commute with $M$, because hopping to the right adds a positive phase (check that) while hopping to the left a negative phase. Reflections break the symmetry in the sign of the phase.

Numeric complex numbers are hard to read, but in exponential terms, the only non vanishing entries in the commutator $[R, M]$ are therefore proportional to $$ -\exp(-\ii) + \exp(\ii) \approx 1.68 \ii,$$ which is precisely the difference in phase when hopping left or right.

In [13]:
R = Reflection_Matrix
#TOOO  adjust space evenly for widget output
comm_RH_widget = widgets.interactive(Show_Commutator,
                            A=fixed(H),
                            B=fixed(R),
                            name_A=fixed("H"),
                            name_B=fixed("R"),
                            phase=phase_Dropdown,
                            n=n_Slider,
                            precision=precision_Slider,
                            axis=axis_Slider,
                        );

comm_RM_widget = widgets.interactive(Show_Commutator,
                            A=fixed(R),
                            B=fixed(M),
                            name_A=fixed("R"),
                            name_B=fixed("M"),
                            n=n_Slider,
                            precision=precision_Slider,
                            axis=axis_Slider,
                        );

display(HBox([VBox([phase_Text_Box, comm_RH_widget]),
             VBox([comm_RM_widget])]))

## Properties of Reflection Matrices

* Reflection matrices $R$ are examples of [involutory matrices](https://en.wikipedia.org/wiki/Involutory_matrix) meaning they are there own inverse, i.e. $RR = \mathbb{1}$.
* They are also symmetric, which implies they are unitary (to be specific: orthogonal, because they are purely real) matrices.
* Hence, the eigenvalues of reflection matrices are $\pm 1$.
* Reflection matrices are generally **not commutative** for $n \geq 3$. Moreover the action of two reflections results in a rotation!

The code below calculates the commutator of two reflection matrices as well as the resulting linear operator when applying two reflections. Compare the resulting matrix with the rotation matrices from above.

In [33]:
from Module_Symmetry_and_Gauge import Show_Commutator_2, Show_R1_R2
from Module_Widgets_and_Sliders import axis2_Slider
#TOOO  adjust space evenly for widget output
comm_RR_widget = widgets.interactive(Show_Commutator_2,
                            A=fixed(R),
                            B=fixed(R),
                            name_A=fixed("R1"),
                            name_B=fixed("R2"),
                            axis1=axis_Slider,
                            axis2=axis2_Slider,
                            n=n_Slider,
                            #precision=precision_Slider,
                        );

R1R2_widget = widgets.interactive(Show_R1_R2,
                                  R1 = fixed(R),
                                  R2 = fixed(R),
                                  n=n_Slider,
                                  axis1=axis_Slider,
                                  axis2=axis2_Slider,
                                 );

display(HBox([VBox([comm_RR_widget]),
              VBox([R1R2_widget])]))

# The Dihedral group $D_n$

An attentive reader will have already figured out, reflections alone are not a closed set under matrix multiplication $ \implies$ they do not form a group (for $ n \geq 3 $). Nevertheless, if one includes the set of rotations we end up with a  valid group. The group of roations and reflections of regular $n$-gons, which is called the [Dihedral group $D_n$](https://en.wikipedia.org/wiki/Dihedral_group#Definition). The Cyclic group $C_n$ is a proper Subgroup of $D_n$, that is $C_n \subset D_n$. This group is non-abelian as we saw that reflections (generally) do not commute and yield rotations. Furthermore reflections and rotations also do not commute and give reflections again.

In the code below you can play around with different rotations and reflections, look at the commutator and what kind of transformation originates from a combination of both.

In [34]:
#TODO add swithc to change order of T and R

In [37]:
#TOOO  adjust space evenly for widget output
comm_RT_widget = widgets.interactive(Show_Commutator,
                            A=fixed(R),
                            B=fixed(T),
                            name_A=fixed("R"),
                            name_B=fixed("T"),
                            n=n_Slider,
                            axis=axis_Slider,
                            turns=turns_Slider,
                            #precision=precision_Slider,
                        );

RT_widget = widgets.interactive(lambda R, T, **kwargs: print("R T = ", R(**kwargs) @ T(**kwargs), sep ="\n"),
                                  R = fixed(T),
                                  T = fixed(R),
                                  n=n_Slider,
                                  axis=axis_Slider,
                                  turns=turns_Slider,
                                 );

display(HBox([VBox([comm_RT_widget]),
              VBox([RT_widget])]))

In [44]:
print(R @ P - P @ R)

NameError: name 'P' is not defined

In [23]:
np.linalg.matrix_power(R,4)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [25]:
R

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [124]:
np.linalg.eigvals(P)

array([-1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j,  0.5+0.8660254j,
        0.5-0.8660254j,  1. +0.j       ])

In [125]:
_, pvec = np.linalg.eig(P)

In [130]:
np.round(pvec.T.conj() @ H @ pvec,0).real

array([[-2.,  0.,  0., -0., -0., -0.],
       [ 0., -1.,  0.,  0., -0.,  0.],
       [ 0.,  0., -1., -0.,  0.,  0.],
       [-0.,  0., -0.,  1.,  0., -0.],
       [-0., -0.,  0.,  0.,  1., -0.],
       [-0.,  0.,  0., -0., -0.,  2.]])

In [138]:
import test_import 


In [71]:
phase_dict = {
    2:[[1,2], [0,2], [-1, 0.45], [np.pi/2, -np.pi/2]],
    3:[[1,2,3], [0,2,4], [-1,5,-0.45], [np.pi/2, -np.pi/2, np.pi/2]],
    4:[[1,2,3,4], [0,2,4,8], [-1,5,-2,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2]],
    5:[[1,2,3,4,5], [0,2,4,6,8], [-1,5,-2,1.5,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2]],
    6:[[1,2,3,4,5,6], [0,2,4,6,8,10], [-1,5,-2,1.5,-0.83,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi]],
    7:[[1,2,3,4,5,6,7], [0,2,4,6,8,10,12], [-1,5,-2,1.5,-0.83,0.7,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2]],
    8:[[1,2,3,4,5,6,7,8], [0,2,4,6,8,10,12,14], [-1,5,-2,1.5,-0.83,0.7,-2,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi]],
    9:[[1,2,3,4,5,6,7,8,9], [0,2,4,6,8,10,12,14,16], [-1,5,-2,1.5,-0.83,0.7,4,-1,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2]],
    10:[[1,2,3,4,5,6,7,8,9,10], [0,2,4,6,8,10,12,14,16,18], [-1,5,-2,1.5,-0.83,0.7,1,-3,-10,0.45], [np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi/2, np.pi/2, -np.pi]]
    }



In [91]:
a = [(f"{list(np.round(i, 2))}", i) for i in phase_dict[6]]
test = widgets.Dropdown()

In [92]:
test.options = a

In [93]:
test

Dropdown(options=(('[1, 2, 3, 4, 5, 6]', [1, 2, 3, 4, 5, 6]), ('[0, 2, 4, 6, 8, 10]', [0, 2, 4, 6, 8, 10]), ('…